In [86]:
from pydantic import BaseModel
from typing import List, Optional
from langchain.schema import BaseMessage, HumanMessage, Document
from langgraph.graph import StateGraph, END, MessagesState
from typing import Annotated, Document
from langgraph.graph import add_messages, StateGraph, END

class SupervisorState(MessagesState):
    """State for the multi-agent system"""
    messages = Annotated[List,add_messages]
    next_agent: str = ""
    Images_data: str = ""
    scientific_data: str =""
    products_data: str = ""
    final_answer: str = ""
    task_complete: bool = False
    current_task: str = ""


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Load the PDF file
pdf_path = "hair_care_scientific.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Size of each chunk
    chunk_overlap=100  # Overlap between chunks
)
chunks = text_splitter.split_documents(documents)

# Generate embeddings
embedding_function = OpenAIEmbeddings()
vector_store = FAISS.from_documents(chunks, embedding_function)

# Save the FAISS index locally
faiss_index_path = "hair_care_faiss_index"
vector_store.save_local(faiss_index_path)

print(f"FAISS index saved locally at {faiss_index_path}")

FAISS index saved locally at hair_care_faiss_index


In [11]:
new_db = FAISS.load_local("hair_care_faiss_index",embedding_function ,allow_dangerous_deserialization=True)

In [12]:
new_retriever = new_db.as_retriever(search_type="mmr", search_kwargs = {"k": 3})

In [13]:
new_retriever.invoke("What are the key factors to consider for maintaining healthy hair?")

[Document(id='0f24f292-97e3-4c4b-8ebe-ab242f2bbf30', metadata={'producer': 'Acrobat Distiller 6.0.1 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2006-02-09T10:30:37+05:30', 'author': 'CLAUDE BOUILLON, JOHN WILKINSON', 'moddate': '2008-04-15T22:43:04-05:00', 'title': 'THE SCIENCE OF HAIR CARE', 'source': 'hair_care_scientific.pdf', 'total_pages': 816, 'page': 8, 'page_label': 'viii'}, page_content='Another recent development is the intens ified awareness of the importance of \nappearance in enhancing the quality of life. Numerous studies by sociologists and \npsychologists have demonstrated that those who are deemed attractive fare better in all \nhuman interactions. The unattractive are at a disadvantage at every stage of the life cycle. \nFrom time immemorial, hair has been celebrated as the “crowning glory” of human \nbeauty. Modern industry has been especially ingenious and innovative in creating \nproducts designed for the scalp and hair, not only for beautif

In [14]:
from typing import TypedDict, Annotated, List, Literal, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
import random
from datetime import datetime

In [92]:
def scientific_data_agent(state: SupervisorState) -> Dict:
    """Agent responsible for retrieving relevant scientific data from the vector database."""
    
    task = state.get("current_task", "scientific query")
    
    # Create query prompt
    query_prompt = f"""As a scientific data specialist, retrieve the most relevant information about: {task}\n\n
    Include:\n
    1. Key scientific findings\n
    2. Relevant studies or research\n
    3. Important data points\n
    4. Practical applications\n\n
    Be concise but thorough."""
    
    # Perform similarity search
    query = state.get("messages", [HumanMessage(content="")])[-1].content
    results = new_retriever.invoke(query, k=3)
    
    # Format the retrieved documents
    documents = "\n\n".join([doc.page_content for doc in results])
    
    # Create agent message
    agent_message = f"🔍 Scientific Data Agent: I've retrieved the most relevant data for '{task}'.\n\nKey findings:\n{documents[:500]}..."
    
    return {
        "messages": [AIMessage(content=agent_message)],
        "scientific_data": documents,
        "next_agent": "supervisor"
    }

In [ ]:
# Test the scientific_data_agent function
if __name__ == "__main__":
    # Create a mock SupervisorState
    mock_state = SupervisorState()

    # Call the scientific_data_agent function
    result = scientific_data_agent(mock_state)

    # Print the results
    print("Agent Messages:")
    for message in result["messages"]:
        print(message.content)

    print("\nRetrieved Scientific Data:")
    print(result["scientific_data"])

    print("\nNext Agent:")
    print(result["next_agent"])
    print(result["messages"])
    print(result[])


Agent Messages:
🔍 Scientific Data Agent: I've retrieved the most relevant data for 'scientific query'.

Key findings:
The science of hair care     396

filled it with barley and wheat and went out with his load. She asked him: 
“What does your burden weigh?” He answered: “Three sacks of wheat, 
two of barley, five in all: this is wh at I have on my shoulders.” Then the 
woman said: “How strong you are! I admire your strength every day.” She 
wanted to know him in the way that women know men, rose and held 
him, saying: “Come, let us spend an hour  in bed. It will be to your profit, 
for I will make you beautifu...

Retrieved Scientific Data:
The science of hair care     396

filled it with barley and wheat and went out with his load. She asked him: 
“What does your burden weigh?” He answered: “Three sacks of wheat, 
two of barley, five in all: this is wh at I have on my shoulders.” Then the 
woman said: “How strong you are! I admire your strength every day.” She 
wanted to know him in 

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

def supervisor_agent(state: SupervisorState) -> Dict:
    """Supervisor agent to control the flow of the multi-agent system."""

    # Retrieve the current task and user messages
    current_task = state.get("current_task", "")
    messages = state.get("messages", [])

    # Check if the user has uploaded a photo
    if current_task == "ask_for_photo":
        user_response = messages[-1].content.lower()
        if "yes" in user_response:
            state["next_agent"] = "image_analysis_agent"
            return {
                "messages": [
                    AIMessage(content="Please upload a photo for analysis."),
                ],
                "next_agent": "image_analysis_agent",
            }
        else:
            state["next_agent"] = "scientific_data_agent"
            return {
                "messages": [
                    AIMessage(content="Proceeding to retrieve scientific data."),
                ],
                "next_agent": "scientific_data_agent",
            }

    # After image analysis, proceed to scientific data retrieval
    if current_task == "image_analysis_complete":
        state["next_agent"] = "scientific_data_agent"
        return {
            "messages": [
                AIMessage(content="Image analysis complete. Now retrieving scientific data."),
            ],
            "next_agent": "scientific_data_agent",
        }

    # After scientific data retrieval, proceed to product information
    if current_task == "scientific_data_complete":
        state["next_agent"] = "product_data_agent"
        return {
            "messages": [
                AIMessage(content="Scientific data retrieved. Now fetching product information."),
            ],
            "next_agent": "product_data_agent",
        }

    # After product information, decide whether to generate the final answer
    if current_task == "product_data_complete":
        # Example decision-making logic
        if state.get("final_answer", ""):
            state["next_agent"] = "final_answer_agent"
            return {
                "messages": [
                    AIMessage(content="All data collected. Generating the final answer."),
                ],
                "next_agent": "final_answer_agent",
            }
        else:
            state["next_agent"] = "scientific_data_agent"  # Example fallback
            return {
                "messages": [
                    AIMessage(content="Revisiting scientific data retrieval for more insights."),
                ],
                "next_agent": "scientific_data_agent",
            }

    # Default behavior: Ask the user if they want to upload a photo
    state["next_agent"] = "supervisor"
    state["current_task"] = "ask_for_photo"
    return {
        "messages": [
            AIMessage(content="Would you like to upload a photo for analysis? (yes/no)"),
        ],
        "next_agent": "supervisor",
    }

In [78]:
from google import genai

In [94]:
from google import genai

def image_analysis_agent(state: SupervisorState) -> Dict:
    """Agent responsible for analyzing an image and describing the hair type."""

    # Extract the image path from the state (assuming it's stored as a file path)
    image_path = "test.png"  # Replace with dynamic retrieval from state if needed

    # Initialize the Google Generative AI client
    client = genai.Client()

    # Upload the image file
    my_file = client.files.upload(file=image_path)

    # Prepare the request with the uploaded file and prompt
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[
            my_file,
            "You are an expert in hair care. Analyze the provided image and describe the hair type.\n\nPossible hair types are:\n- Heavily Damaged & Dry\n- Colored & Bleached\n- Heavily Damaged\n- Damaged & Dry\n- Dry, Damaged\n- Dry & Colored\n- Damaged & Bleached\n- Strawy, Damaged\n- Brittle & Dull\n- Prone to Split Ends\n- Normal to Dry\n- Fine & Slightly Dry\n- Normal & Fine\n- Long hair with greasy roots\n- Long hair, fine & normal\n- Long hair with dry tips\n\nProvide a brief description of the hair type and any additional observations."
        ]
    )

    # Update the state with the analysis
    state["Images_data"] = response.text

    # Create agent message
    agent_message = f"🖼️ Image Analysis Agent: I've analyzed the image.\n\nHair Type: {response.text}"

    return {
        "messages": [AIMessage(content=agent_message)],
        "Images_data": response.text,
        "next_agent": "supervisor"
    }

In [95]:
# Test the image_analysis_agent function
if __name__ == "__main__":
    # Create a mock SupervisorState
    mock_state = SupervisorState()

    # Call the image_analysis_agent function
    result = image_analysis_agent(mock_state)

    # Print the results
    print("Agent Messages:")
    for message in result["messages"]:
        print(message.content)

    print("\nAnalyzed Image Data:")
    print(result["Images_data"])

    print("\nNext Agent:")

    print(result["next_agent"])

Agent Messages:
🖼️ Image Analysis Agent: I've analyzed the image.

Hair Type: Based on the provided image, the hair type is best described as:

**Dry & Colored**

**Description and Observations:**
The image clearly shows significant grey regrowth at the roots, indicating that the hair has been colored. The darker hair strands, as well as the grey ones, appear to lack natural luster and moisture, presenting a somewhat dull and dry appearance. There's a slight frizziness, particularly noticeable around the hairline and at the roots where the grey is visible, which is often indicative of dryness. While not appearing heavily damaged with severe breakage, the hair shows signs of dehydration, likely exacerbated by the coloring process.

Analyzed Image Data:
Based on the provided image, the hair type is best described as:

**Dry & Colored**

**Description and Observations:**
The image clearly shows significant grey regrowth at the roots, indicating that the hair has been colored. The darker h

In [101]:
from langchain_google_genai import ChatGoogleGenerativeAI

def final_answer_agent(state: SupervisorState) -> Dict:
    """Agent responsible for generating the final answer based on products_data, scientific_data, and image description."""

    # Retrieve necessary data from the state
    products_data = state.get("products_data", "No product data available.")
    scientific_data = state.get("scientific_data", "No scientific data available.")
    image_description = state.get("Images_data", "No image description available.")

    # Combine the data into a comprehensive answer
    combined_prompt = (
        f"Based on the analysis, generate a comprehensive answer using the following data:\n\n"
        f"🔬 Scientific Data:\n{scientific_data}\n\n"
        f"🛍️ Product Recommendations:\n{products_data}\n\n"
        f"🖼️ Image Analysis:\n{image_description}\n\n"
        f"Provide a brief, clear and user-friendly response."
    )

    # Initialize the LLM
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

    # Generate the final answer using the LLM
    llm_response = llm.invoke(combined_prompt)

    # Update the state with the final answer
    state["final_answer"] = llm_response.content
    state["task_complete"] = True

    # Create agent message
    agent_message = f"✅ Final Answer Agent: The task is complete. Here is the final answer:\n\n{llm_response.content}"

    return {
        "messages": [AIMessage(content=agent_message)],
        "final_answer": llm_response.content,
        "task_complete": True
    }


In [102]:
# Test the final_answer_agent function
if __name__ == "__main__":
    # Create a mock SupervisorState
    mock_state = SupervisorState(
        next_agent="final_answer_agent",
        products_data="Recommended Product: Sulfate-Free Shampoo",
        scientific_data="Scientific studies show that sulfate-free shampoos reduce scalp irritation and preserve natural oils.",
        Images_data="Hair Type: Fine & Slightly Dry",
        messages=[HumanMessage(content="What is the best recommendation based on the analysis?")],
    )

    # Call the final_answer_agent function
    result = final_answer_agent(mock_state)

    # Print the results
    print("Agent Messages:")
    for message in result["messages"]:
        print(message.content)

    print("\nFinal Answer:")
    print(result["final_answer"])

    print("\nTask Complete:")
    print(result["task_complete"])

Agent Messages:
✅ Final Answer Agent: The task is complete. Here is the final answer:

For your fine and slightly dry hair, we recommend using a **Sulfate-Free Shampoo**. Scientific studies show that these shampoos reduce scalp irritation and help preserve your hair's natural oils, which is ideal for maintaining moisture and preventing further dryness in your hair type.

Final Answer:
For your fine and slightly dry hair, we recommend using a **Sulfate-Free Shampoo**. Scientific studies show that these shampoos reduce scalp irritation and help preserve your hair's natural oils, which is ideal for maintaining moisture and preventing further dryness in your hair type.

Task Complete:
True
